<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/Price/prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data =pd.read_csv('/content/drive/MyDrive/AIEngineer/estimator-med-cost/ahhh.csv')
data.head()

,cid,patient,illnes_type,chief_complaint,vital_sign,billing,diagnosis,total_bill_amount
0,3101202037015,"{""hn"": ""45-10056"", ""dob"": ""1966-06-30"", ""gende...",ACC,ขาขวากระแทกประตู 09.30 น \r\nไม่แพ้ยา,"[{""heart_rate"": ""76"", ""pain_score"": ""0"", ""syst...","[{""billing_initial"": ""1032.00"", ""billing_disco...","[{""icd10"": ""S711"", ""dx_name"": ""Open wound of t...",4358.0
1,1102001128174,"{""hn"": ""692775"", ""dob"": ""1989-04-15"", ""gender""...",ILL,เจ็บคอ 4 d PTA,"[{""heart_rate"": ""80"", ""pain_score"": ""0"", ""syst...","[{""billing_initial"": ""975.00"", ""billing_discou...","[{""icd10"": ""J00"", ""dx_name"": ""Acute nasopharyn...",1728.0
2,1640100085281,"{""hn"": ""66-19861"", ""dob"": ""1988-04-25"", ""gende...",ILL,เจ็บคอ,"[{""heart_rate"": ""82"", ""pain_score"": ""0"", ""syst...","[{""billing_initial"": ""700"", ""billing_discount""...","[{""icd10"": ""J029"", ""dx_name"": ""Acute pharyngit...",2014.0
3,1103100256856,"{""hn"": ""59-42311"", ""dob"": ""1994-06-09"", ""gende...",ILL,ปวดท้องประจำเดือน,"[{""heart_rate"": ""80"", ""pain_score"": ""0"", ""syst...","[{""billing_initial"": ""500"", ""billing_discount""...","[{""icd10"": ""N944"", ""dx_name"": ""Primary dysmeno...",1410.0
4,1101499023411,"{""hn"": ""55-33689"", ""dob"": ""1986-03-11"", ""gende...",ILL,"ไอ เจ็บคอ /w,jcrhpk","[{""heart_rate"": ""92"", ""pain_score"": ""0"", ""syst...","[{""billing_initial"": ""1436.00"", ""billing_disco...","[{""icd10"": ""J069"", ""dx_name"": ""Acute upper res...",2463.0


In [2]:
data['patient'][0]

'{"hn": "45-10056", "dob": "1966-06-30", "gender": "M", "last_name": "วงศ์สถิตย์พร", "first_name": "สุพจน์", "title_name": "นาย", "patient_registration_id": "2c1eb4b7-3566-4462-bd82-53fa4d6d08f4"}'

In [3]:
data['vital_sign'][0]

'[{"heart_rate": "76", "pain_score": "0", "systolic_bp": "156.00", "temperature": "36.60", "diastolic_bp": "82.00", "respiratory_rate": "20", "oxygen_saturation": "99.00", "vital_sign_entry_datetime": "2024-08-31 15:58"}]'

In [27]:
import ast

df = pd.DataFrame(data)

# Preprocessing patient column
def extract_patient_info(patient_str):
    patient_data = ast.literal_eval(patient_str)
    return pd.Series({
        "dob": patient_data.get("dob"),
        "gender": patient_data.get("gender")
    })

patient_info = df["patient"].apply(extract_patient_info)
df = pd.concat([df, patient_info], axis=1)

def extract_vital_signs(vital_sign_str):
    vital_data = ast.literal_eval(vital_sign_str)[0]  # Convert string to list of dicts and take the first dict

    def clean_float(value):
        try:
            # Strip whitespace and convert to float
            return float(str(value).strip()) if str(value).strip() != '' else 0.0
        except (ValueError, TypeError):
            # Return 0.0 for invalid or missing values
            return 0.0

    return pd.Series({
        "heart_rate": clean_float(vital_data.get("heart_rate", 0)),
        "pain_score": clean_float(vital_data.get("pain_score", 0)),
        "systolic_bp": clean_float(vital_data.get("systolic_bp", 0)),
        "diastolic_bp": clean_float(vital_data.get("diastolic_bp", 0)),
        "temperature": clean_float(vital_data.get("temperature", 0)),
        "respiratory_rate": clean_float(vital_data.get("respiratory_rate", 0)),
        "oxygen_saturation": clean_float(vital_data.get("oxygen_saturation", 0)),
        "vital_sign_entry_datetime": pd.to_datetime(vital_data.get("vital_sign_entry_datetime"), errors='coerce')
    })

vital_signs = df["vital_sign"].apply(extract_vital_signs)
df = pd.concat([df, vital_signs], axis=1)

In [28]:
from datetime import datetime

def calculate_age(dob_str):
    dob = datetime.strptime(dob_str, "%Y-%m-%d")
    today = datetime.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

df["age"] = df["dob"].apply(calculate_age)

In [29]:
mapping = {
    "ILL": 1,
    "ACC": 2,
    "ER": 3,
    "FU": 4,
    "DEN": 5,
    "HD": 6
}
gender = {
    "F": 0,
    "M": 1
}
df["illnes_type"] = df["illnes_type"].map(mapping)
df["gender"] = df["gender"].map(gender)
df.rename(columns={"illnes_type": "illness_type"}, inplace=True)

In [18]:
df.columns

Index(['cid', 'patient', 'illness_type', 'chief_complaint', 'vital_sign',
       'billing', 'diagnosis', 'total_bill_amount', 'hn', 'dob', 'gender',
       'heart_rate', 'pain_score', 'systolic_bp', 'diastolic_bp',
       'temperature', 'respiratory_rate', 'oxygen_saturation',
       'vital_sign_entry_datetime', 'age'],
      dtype='object')

In [33]:
df = df.drop(columns=['patient', 'vital_sign', 'dob', 'billing'])

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14887 entries, 0 to 14886
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   cid                        14887 non-null  object        
 1   illness_type               14887 non-null  int64         
 2   chief_complaint            14887 non-null  object        
 3   diagnosis                  14887 non-null  object        
 4   total_bill_amount          14887 non-null  float64       
 5   gender                     14887 non-null  int64         
 6   heart_rate                 14887 non-null  float64       
 7   pain_score                 14201 non-null  float64       
 8   systolic_bp                14887 non-null  float64       
 9   diastolic_bp               14887 non-null  float64       
 10  temperature                14887 non-null  float64       
 11  respiratory_rate           14887 non-null  float64       
 12  oxyg

In [36]:
df['diagnosis'][0]

'[{"icd10": "S711", "dx_name": "Open wound of thigh", "dx_type": "PP"}, {"icd10": "W220", "dx_name": "Striking against or struck by other object(s)\xa0: at home", "dx_type": "EC"}]'

In [65]:
df.head()

,cid,illness_type,chief_complaint,diagnosis,total_bill_amount,gender,heart_rate,pain_score,systolic_bp,diastolic_bp,temperature,respiratory_rate,oxygen_saturation,vital_sign_entry_datetime,age
0,3101202037015,2,ขาขวากระแทกประตู 09.30 น \r\nไม่แพ้ยา,"[{""icd10"": ""S711"", ""dx_name"": ""Open wound of t...",4358.0,1,76.0,0.0,156.0,82.0,36.6,20.0,99.0,2024-08-31 15:58:00,58
1,1102001128174,1,เจ็บคอ 4 d PTA,"[{""icd10"": ""J00"", ""dx_name"": ""Acute nasopharyn...",1728.0,0,80.0,0.0,119.0,90.0,36.9,20.0,99.0,2024-08-08 19:41:00,35
2,1640100085281,1,เจ็บคอ,"[{""icd10"": ""J029"", ""dx_name"": ""Acute pharyngit...",2014.0,1,82.0,0.0,114.0,70.0,36.7,20.0,96.0,2024-08-31 15:40:00,36
3,1103100256856,1,ปวดท้องประจำเดือน,"[{""icd10"": ""N944"", ""dx_name"": ""Primary dysmeno...",1410.0,0,80.0,0.0,116.0,68.0,36.6,20.0,100.0,2024-08-10 17:02:00,30
4,1101499023411,1,"ไอ เจ็บคอ /w,jcrhpk","[{""icd10"": ""J069"", ""dx_name"": ""Acute upper res...",2463.0,0,92.0,0.0,133.0,53.0,36.6,20.0,98.0,2024-08-10 18:51:00,38


In [67]:
opd.columns

Index(['transaction_uid', 'transaction_no', 'hospital_code', 'hospital_name',
       'visit_date', 'dx_free_text', 'claim_status', 'coma_score',
       'discharge_at', 'approve_at', 'icd9', 'vital_sign', 'billing',
       'order_item', 'icd_10', 'policy_detail'],
      dtype='object')

In [72]:
opd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   transaction_uid  150000 non-null  object
 1   transaction_no   150000 non-null  object
 2   hospital_code    150000 non-null  object
 3   hospital_name    150000 non-null  object
 4   visit_date       150000 non-null  object
 5   dx_free_text     150000 non-null  object
 6   claim_status     150000 non-null  object
 7   coma_score       150000 non-null  object
 8   discharge_at     150000 non-null  object
 9   approve_at       150000 non-null  object
 10  icd9             150000 non-null  object
 11  vital_sign       150000 non-null  object
 12  billing          150000 non-null  object
 13  order_item       150000 non-null  object
 14  icd_10           150000 non-null  object
 15  policy_detail    150000 non-null  object
dtypes: object(16)
memory usage: 18.3+ MB


In [78]:
opd[opd['vital_sign'] != '[]'][['hospital_name','visit_date','dx_free_text','vital_sign','icd_10','billing']]

,hospital_name,visit_date,dx_free_text,vital_sign,icd_10,billing
0,โรงพยาบาลราชเวชอุบลราชธานี,2025-01-13 23:35,ล้ม,[],"[{'code': 'S808', 'dx_name': 'Other superficia...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
1,โรงพยาบาลจุฬารัตน์ 3 อินเตอร์,2025-01-13 23:05,-,[],"[{'code': 'J069', 'dx_name': 'Acute upper resp...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
2,โรงพยาบาลวิชัยเวช อินเตอร์เนชั่นแนล สมุทรสาคร,2025-01-13 18:20,ชาเส้นประสาท,[],"[{'code': 'E511', 'dx_name': 'Beriberi', 'dx_t...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
3,โรงพยาบาลนครธน,2025-01-13 23:05,แมวข่วนใบหน้า เหตุเกิด 12/1/68 เวลา 22.00 น. ท...,[],"[{'code': 'S014', 'dx_name': 'Open wound of ch...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
4,โรงพยาบาลวิชัยเวช อินเตอร์เนชั่นแนล สมุทรสาคร,2025-01-13 18:40,ฝุ่นเข้าตา ตาเจ็บ,[],"[{'code': 'B308', 'dx_name': 'Other viral conj...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
...,...,...,...,...,...,...
149995,โรงพยาบาลเกษมราษฎร์ รัตนาธิเบศร์,2024-10-26 17:05,ติดตามอาการ,[],"[{'code': 'K219', 'dx_name': 'Gastro-oesophage...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
149996,โรงพยาบาลสำโรงการแพทย์,2024-10-26 16:50,ไซนัสอักเสบ,"[{'diastolic_bp': '', 'systolic_bp': '', 'hear...","[{'code': 'J010', 'dx_name': 'Acute maxillary ...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
149997,โรงพยาบาลวิมุต,2024-10-26 16:50,.,[],"[{'code': 'S911', 'dx_name': 'Open wound of to...","[{'simb_code': '1.1.14(2)', 'simb_description'..."
149998,โรงพยาบาลเกษมราษฎร์อินเตอร์เนชั่นแนล รัตนาธิเบศร์,2024-10-26 16:50,ลอดลมอักเสบเฉียบพลัน ที่มิได้ระบุรายละเอียด,"[{'diastolic_bp': '52.00', 'systolic_bp': '96....","[{'code': 'J209', 'dx_name': 'Acute bronchitis...","[{'simb_code': '1.1.1(3)', 'simb_description':..."
